In [ ]:
import os, urllib.request, json, shapefile, shutil, subprocess, csv, fiona, shapely, geopandas, psycopg2, pandas
from osgeo import gdal, ogr, osr
from datetime import datetime

In [ ]:
os.environ['SHAPE_ENCODING'] = "utf-8"
basePath = 'C:\\saprog\\projeto'
outputPath = 'C:\\saprog\\projeto\\output'
os.chdir(basePath)

def deleteAndCreateOutput():
    print ('A limpar e criar directoria de outputs...')
    outDir = os.path.join(basePath, 'output')
    if os.path.isdir(outDir):
        shutil.rmtree(outDir)
    os.mkdir(outDir)

deleteAndCreateOutput()

In [ ]:
print('A ler shapefile...')
caop = geopandas.read_file('caop.shp',encoding='utf-8')
# caop.plot()
# caop.head()
# gdf = caop.loc[caop['Concelho'] == 'Bombarral']
# gdf.head()

In [ ]:
print('A executar Dissolve...')
distritos = caop.dissolve(by = 'Distrito')
# distritos.plot()
# distritos.head(1)

In [ ]:
print('A extrair centroides das regiões admninistrativas...')
shpOutput = os.path.join(outputPath, 'centroides.shp')

# dataProj = distritos.to_crs(epsg = 3763)
# centroides = dataProj.centroid
centroides = distritos.centroid
centroides.to_file(shpOutput, encoding='utf-8')
# centroides.plot()
# centroides.head()

In [ ]:
coordx = centroides.x.to_dict()
coordy = centroides.y.to_dict()
# print(coordx)
# print(coordy)

def getCoordinates(dicX, dicY):
    coord = [dicX, dicY]
    coordDic = {}
    for i in dicX.keys():
        coordDic[i] = tuple(coordDic[i] for coordDic in coord)
    return coordDic

print('A extrair coodenadas geográficas...')
coord = getCoordinates(coordx, coordy)

In [ ]:
# Carregamento de shapefile na bd
# command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
#           "-f", "PostgreSQL",
#           "PG:host=localhost user=postgres dbname=meteo password=3763",
#           shpOutput,
#           "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
#           " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "centroides", "-overwrite"]
# subprocess.check_call(command)

In [ ]:
apiKey = '44cfad7f82ec61d3f420de3201b703d4'

def getWeather(coordDic, apikey):
    forecast = []
    for item in coord.items():
        lat = str(item[1][1])
        long = str(item[1][0])
        url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + long + \
            '&exclude=minutely,hourly,daily,alerts&appid=' + apiKey + '&units=metric'
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            districtForecast = {}
            districtForecast['distrito'] = item[0]
            districtForecast['date'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%d-%m-%Y')
            districtForecast['time'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%H:%M:%S')
            main = data.get('current').get('weather')
            for item in main:
                districtForecast['weather_desc'] = item.get('main')
            districtForecast['temperature'] = data.get('current').get('temp')
            districtForecast['feels_like'] = data.get('current').get('feels_like')
            districtForecast['pressure'] = data.get('current').get('pressure')
            districtForecast['humidity'] = data.get('current').get('humidity')
            districtForecast['dew_point'] = data.get('current').get('dew_point')
            districtForecast['ultrav_index'] = data.get('current').get('uvi')
            districtForecast['wind_speed'] = data.get('current').get('wind_speed')
            districtForecast['wind_deg'] = data.get('current').get('wind_deg')


            forecast.append(districtForecast)
    return forecast

df = pandas.DataFrame(getWeather(coord, apiKey))
df.head(-1)

In [ ]:
#